In [1]:
import kagglehub
import os

os.environ['KAGGLEHUB_CACHE'] = 'E:\\Datasets'
kagglehub.dataset_download("gauravduttakiit/resume-dataset")

D:\Python\python3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'E:\\Datasets\\datasets\\gauravduttakiit\\resume-dataset\\versions\\1'

In [2]:
import torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [5]:
df = pd.read_csv('E:\\Datasets\\datasets\\gauravduttakiit\\resume-dataset\\versions\\1\\UpdatedResumeDataSet.csv')

In [6]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [7]:
df.describe()

,Category,Resume
count,962,962
unique,25,166
top,Java Developer,"Technical Skills Web Technologies: Angular JS,..."
freq,84,18


In [8]:
df.iloc[0,1]

'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigatio

In [68]:
from bs4 import BeautifulSoup
import re
import json

In [10]:
def clean_cv(text):
    text = BeautifulSoup(text, 'html.parser').get_text()

    text = re.sub(r'[\r\n\t]', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)    
    text = re.sub(r'\s+', ' ', text).strip()

    text = text.lower()

    return text

In [11]:
df['Cleaned_Resume'] = df['Resume'].apply(clean_cv)

In [12]:
df.head()

,Category,Resume,Cleaned_Resume
0,Data Science,Skills * Programming Languages: Python (pandas...,skills * programming languages: python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may 2013 to may 2017 b.e uit...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...","areas of interest deep learning, control syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills r python sap hana tableau sap hana sql ...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...","education details mca ymcaust, faridabad, hary..."


In [13]:
df.iloc[0,2]

'skills * programming languages: python (pandas, numpy, scipy, scikit-learn, matplotlib), sql, java, javascript/jquery. * machine learning: regression, svm, na ve bayes, knn, random forest, decision trees, boosting techniques, cluster analysis, word embedding, sentiment analysis, natural language processing, dimensionality reduction, topic modelling (lda, nmf), pca & neural nets. * database visualizations: mysql, sqlserver, cassandra, hbase, elasticsearch d3.js, dc.js, plotly, kibana, matplotlib, ggplot, tableau. * others: regular expression, html, css, angular 6, logstash, kafka, python flask, git, docker, computer vision - open cv and understanding of deep learning.education details data science assurance associate data science assurance associate - ernst & young llp skill details javascript- exprience - 24 months jquery- exprience - 24 months python- exprience - 24 monthscompany details company - ernst & young llp description - fraud investigations and dispute services assurance tec

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [55]:
def build_few_shot_prompt(resume_text):
    return f"""
You are a structured resume parsing agent.

## INSTRUCTIONS:
Extract the following fields from the resume. Respond only with a valid JSON object, using double quotes. Do not guess.

- "experience_years": Float. Sum of relevant work experience in years. Use months/12 if stated. If unclear, use 0.0.
- "technical_skills": List (max 10). Tools, languages, or frameworks. All lowercase.
- "soft_skills": List (max 3). Interpersonal or cognitive skills. No technical terms. All lowercase.
- "education_level": One of ["bachelor", "master", "phd", "other"].
- "fit_score": Float [0.0–5.0], based on:
  - experience (0–2)
  - technical skill breadth (0–2)
  - soft skills (0–1)

## EXAMPLE RESUME:
education: b.e in computer engineering, 2017. skills: python, java, sql, scikit-learn. worked 18 months as data analyst at ABC corp. known for team collaboration and adaptability.

## EXAMPLE OUTPUT:
{{
  "experience_years": 1.5,
  "technical_skills": ["python", "java", "sql", "scikit-learn"],
  "soft_skills": ["collaboration", "adaptability"],
  "education_level": "bachelor",
  "fit_score": 3.7
}}

## EXAMPLE RESUME:
education: masters in cs, 2014. skills: python, excel, databricks, scikit-learn. worked 24 months as data engineer. team player.

## EXAMPLE OUTPUT:
{{
  "experience_years": 2.0,
  "technical_skills": ["python", "excel", "databricks", "scikit-learn"],
  "soft_skills": ["team", "collaboration"],
  "education_level": "master",
  "fit_score": 4.1
}}

## EXAMPLE RESUME:
education: undergraduated cs student, 2019. skills: python, pytorch, keras, tensorflow. internship for 3 months at school. i am proactive, agile, quick-learner student.

## EXAMPLE OUTPUT:
{{
  "experience_years": 0.3,
  "technical_skills": ["python", "pytorch", "keras", "tensorflow"],
  "soft_skills": ["proactive", "agile", "quick learner"],
  "education_level": "other",
  "fit_score": 3.5
}}

### Now analyze the resume below. Don't copy any example. Parse it precisely.
## RESUME:
\"\"\"
{resume_text}
\"\"\"

FINISH ON JSON CLOSING BRACKET "}}"
## OUTPUT:
"""

In [56]:
def build_zero_shot_prompt(resume_text):
    return f"""
You are a structured resume parsing agent.

## INSTRUCTIONS:
Extract the following fields from the resume. Respond only with a valid JSON object, using double quotes. Do not guess.

- "experience_years": Float. Sum of relevant work experience in years. Use months/12 if stated. If unclear, use 0.0.
- "technical_skills": List (max 10). Tools, languages, or frameworks. All lowercase.
- "soft_skills": List (max 3). Interpersonal or cognitive skills. No technical terms. All lowercase.
- "education_level": One of ["bachelor", "master", "phd", "other"].
- "fit_score": Float [0.0–5.0], based on:
  - experience (0–2)
  - technical skill breadth (0–2)
  - soft skills (0–1)

### Now analyze the resume below. Don't copy any example. Parse it precisely.
## RESUME:
\"\"\"
{resume_text}
\"\"\"

FINISH ON JSON CLOSING BRACKET "}}"
## OUTPUT:
"""

In [57]:
df['Cleaned_Resume'].iloc[3,]

"skills r python sap hana tableau sap hana sql sap hana pal ms sql sap lumira c# linear programming data modelling advance analytics scm analytics retail analytics social media analytics nlp education details january 2017 to january 2018 pgdm business analytics great lakes institute of management & illinois institute of technology january 2013 bachelor of engineering electronics and communication bengaluru, karnataka new horizon college of engineering, bangalore visvesvaraya technological university data science consultant consultant - deloitte usi skill details linear programming- exprience - 6 months retail- exprience - 6 months retail marketing- exprience - 6 months scm- exprience - 6 months sql- exprience - less than 1 year months deep learning- exprience - less than 1 year months machine learning- exprience - less than 1 year months python- exprience - less than 1 year months r- exprience - less than 1 year monthscompany details company - deloitte usi description - the project inv

In [67]:
from transformers import pipeline

prompt = build_zero_shot_prompt(df['Cleaned_Resume'].iloc[11,])

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

output = pipe(prompt, max_new_tokens=500, temperature=0.4, top_p=0.8)[0]["generated_text"]
print(output)

Device set to use cuda:0



You are a structured resume parsing agent.

## INSTRUCTIONS:
Extract the following fields from the resume. Respond only with a valid JSON object, using double quotes. Do not guess.

- "experience_years": Float. Sum of relevant work experience in years. Use months/12 if stated. If unclear, use 0.0.
- "technical_skills": List (max 10). Tools, languages, or frameworks. All lowercase.
- "soft_skills": List (max 3). Interpersonal or cognitive skills. No technical terms. All lowercase.
- "education_level": One of ["bachelor", "master", "phd", "other"].
- "fit_score": Float [0.0–5.0], based on:
  - experience (0–2)
  - technical skill breadth (0–2)
  - soft skills (0–1)

### Now analyze the resume below. Don't copy any example. Parse it precisely.
## RESUME:
"""
education details may 2013 to may 2017 b.e uit-rgpv data scientist data scientist - matelabs skill details python- exprience - less than 1 year months statsmodels- exprience - 12 months aws- exprience - less than 1 year months machin

In [77]:
param_pairs = [
    (0.2, 0.7),
    (0.4, 0.8),
    (0.6, 0.9),
    (0.8, 1.0),
    (1.0, 0.95),
    (0.2, 0.3),
    (0.3, 0.45)
]
pattern = r"## OUTPUT:\s*(\{.*\})"
prompt = build_zero_shot_prompt(df['Cleaned_Resume'].iloc[11,])

for idx, (temp, top_p) in enumerate(param_pairs):
    print(f"\n--- Test {idx+1}: temperature={temp}, top_p={top_p} ---")
    output = pipe(
        prompt,
        max_new_tokens=500,
        temperature=temp,
        top_p=top_p
    )[0]["generated_text"]

    match = re.search(pattern, output, re.DOTALL)
    try:
        json_output = match.group(1).strip()
        json_string = json.loads(json_output)
    except Exception as e:
        print(f"ERROR! {e}")
    print(json_string)


--- Test 1: temperature=0.2, top_p=0.7 ---
ERROR! 'NoneType' object has no attribute 'group'
{'experience_years': 52.0, 'technical_skills': ['python', 'math', 'numpy', 'pandas', 'scikit-learn'], 'soft_skills': ['data science', 'machine learning'], 'education_level': 'bachelor', 'fit_score': 3.163397726092923, 'favorite_location': 'India'}

--- Test 2: temperature=0.4, top_p=0.8 ---
ERROR! 'NoneType' object has no attribute 'group'
{'experience_years': 52.0, 'technical_skills': ['python', 'math', 'numpy', 'pandas', 'scikit-learn'], 'soft_skills': ['data science', 'machine learning'], 'education_level': 'bachelor', 'fit_score': 3.163397726092923, 'favorite_location': 'India'}

--- Test 3: temperature=0.6, top_p=0.9 ---
{'experience_years': 4, 'technical_skills': ['python', 'machine learning', 'data science'], 'soft_skills': ['data science', 'machine learning', 'data analysis'], 'education_level': 'bachelor', 'fit_score': 3.0}

--- Test 4: temperature=0.8, top_p=1.0 ---
{'experience_year

In [ ]:
param_pairs = [
    (0.2, 0.7),
    (0.4, 0.8),
    (0.6, 0.9),
    (0.8, 1.0),
    (1.0, 0.95),
    (0.2, 0.3),
    (0.3, 0.45)
]
pattern = r"## OUTPUT:\s*(\{.*\})"
prompt = build_few_shot_prompt(df['Cleaned_Resume'].iloc[11,])

for idx, (temp, top_p) in enumerate(param_pairs):
    print(f"\n--- Test {idx+1}: temperature={temp}, top_p={top_p} ---")
    output = pipe(
        prompt,
        max_new_tokens=500,
        temperature=temp,
        top_p=top_p
    )[0]["generated_text"]

    match = re.search(pattern, output, re.DOTALL)
    try:
        json_output = match.group(1).strip()
        json_string = json.loads(json_output)
    except Exception as e:
        print(f"ERROR! {e}")
    print(json_string)


--- Test 1: temperature=0.2, top_p=0.7 ---


In [18]:
pattern = r"## OUTPUT:\s*(\{.*\})"
match = re.search(pattern, output, re.DOTALL)
try:
    json_output = match.group(1).strip()
    json_string = json.loads(json_output)
except:
    print("ERROR!")

In [19]:
list(json_string.values())

[1.5,
 ['python', 'core java', 'database management'],
 ['collaboration', 'adaptability'],
 'bachelor',
 3.7]

In [20]:
list(json_string.keys())

['experience_years',
 'technical_skills',
 'soft_skills',
 'education_level',
 'fit_score']

In [21]:
obj = df.iloc[1,]
obj.iloc[2]

row = [*obj.tolist()] 
row

['Data Science',
 'Education Details \r\nMay 2013 to May 2017 B.E   UIT-RGPV\r\nData Scientist \r\n\r\nData Scientist - Matelabs\r\nSkill Details \r\nPython- Exprience - Less than 1 year months\r\nStatsmodels- Exprience - 12 months\r\nAWS- Exprience - Less than 1 year months\r\nMachine learning- Exprience - Less than 1 year months\r\nSklearn- Exprience - Less than 1 year months\r\nScipy- Exprience - Less than 1 year months\r\nKeras- Exprience - Less than 1 year monthsCompany Details \r\ncompany - Matelabs\r\ndescription - ML Platform for business professionals, dummies and enthusiasts.\r\n60/A Koramangala 5th block,\r\nAchievements/Tasks behind sukh sagar, Bengaluru,\r\nIndia                               Developed and deployed auto preprocessing steps of machine learning mainly missing value\r\ntreatment, outlier detection, encoding, scaling, feature selection and dimensionality reduction.\r\nDeployed automated classification and regression model.\r\nlinkedin.com/in/aditya-rathore-\r\

In [22]:
import csv

In [23]:
if os.path.exists('cleaned_resume_dataset.csv'):
    os.remove('cleaned_resume_dataset.csv')

    with open('cleaned_resume_dataset.csv', "a", newline="") as f:
        writer = csv.writer(f)
        
        head = [*df.columns[::2], *list(json_string.keys())]
        writer.writerow(head)

for i in range(df.shape[0]):
    resume = df.iloc[i,::2]
    prompt = build_prompt(resume['Cleaned_Resume'])

    output = pipe(prompt, max_new_tokens=500, temperature=0.2)[0]["generated_text"]

    try:
        pattern = r"## OUTPUT:\s*(\{.*\})"
        match = re.search(pattern, output, re.DOTALL)
               
        json_text = match.group(1).strip()
        json_data = json.loads(json_text)
    except Exception as e:
        print(f"ERROR {e}\n{output}\n{'-'*30}")
        continue

    with open('cleaned_resume_dataset.csv', "a", newline="") as f:
        writer = csv.writer(f)
        
        row = [*resume.tolist(), *list(json_data.values())]
        writer.writerow(row)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


ERROR 'NoneType' object has no attribute 'group'

You are a structured resume parsing agent.

## INSTRUCTIONS:
Extract the following fields from the resume. Respond only with a valid JSON object, using double quotes. Do not guess.

- "experience_years": Float. Sum of relevant work experience in years. Use months/12 if stated. If unclear, use 0.0.
- "technical_skills": List (max 10). Tools, languages, or frameworks. All lowercase.
- "soft_skills": List (max 3). Interpersonal or cognitive skills. No technical terms. All lowercase.
- "education_level": One of ["bachelor", "master", "phd", "other"].
- "fit_score": Float [0.0–5.0], based on:
  - experience (0–2)
  - technical skill breadth (0–2)
  - soft skills (0–1)

## EXAMPLE RESUME:
education: b.e in computer engineering, 2017. skills: python, java, sql, scikit-learn. worked 18 months as data analyst at ABC corp. known for team collaboration and adaptability.

## EXAMPLE OUTPUT:
{
  "experience_years": 1.5,
  "technical_skills": ["pytho

Token indices sequence length is longer than the specified maximum sequence length for this model (2120 > 2048). Running this sequence through the model will result in indexing errors


ERROR Expecting ',' delimiter: line 9 column 6 (char 280)

You are a structured resume parsing agent.

## INSTRUCTIONS:
Extract the following fields from the resume. Respond only with a valid JSON object, using double quotes. Do not guess.

- "experience_years": Float. Sum of relevant work experience in years. Use months/12 if stated. If unclear, use 0.0.
- "technical_skills": List (max 10). Tools, languages, or frameworks. All lowercase.
- "soft_skills": List (max 3). Interpersonal or cognitive skills. No technical terms. All lowercase.
- "education_level": One of ["bachelor", "master", "phd", "other"].
- "fit_score": Float [0.0–5.0], based on:
  - experience (0–2)
  - technical skill breadth (0–2)
  - soft skills (0–1)

## EXAMPLE RESUME:
education: b.e in computer engineering, 2017. skills: python, java, sql, scikit-learn. worked 18 months as data analyst at ABC corp. known for team collaboration and adaptability.

## EXAMPLE OUTPUT:
{
  "experience_years": 1.5,
  "technical_skills"

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ERROR 'NoneType' object has no attribute 'group'

You are a structured resume parsing agent.

## INSTRUCTIONS:
Extract the following fields from the resume. Respond only with a valid JSON object, using double quotes. Do not guess.

- "experience_years": Float. Sum of relevant work experience in years. Use months/12 if stated. If unclear, use 0.0.
- "technical_skills": List (max 10). Tools, languages, or frameworks. All lowercase.
- "soft_skills": List (max 3). Interpersonal or cognitive skills. No technical terms. All lowercase.
- "education_level": One of ["bachelor", "master", "phd", "other"].
- "fit_score": Float [0.0–5.0], based on:
  - experience (0–2)
  - technical skill breadth (0–2)
  - soft skills (0–1)

## EXAMPLE RESUME:
education: b.e in computer engineering, 2017. skills: python, java, sql, scikit-learn. worked 18 months as data analyst at ABC corp. known for team collaboration and adaptability.

## EXAMPLE OUTPUT:
{
  "experience_years": 1.5,
  "technical_skills": ["pytho

KeyboardInterrupt: 